# Secure Emails

By Sally Lee and Zara Coakley

When you send an email, you want to know the content of your message is private. Did you know that email isn't end-to-end encrypted, meaning that it's possible for your email provider or outside individuals to intercept and read your email messages? Not cool. One solution to this? Public-key cryptography.

Public-key cryptography uses the concept of public and private keys to encrypt your messages in a way that anyone can send you a message, but only you can read it. We created a script that lets you send an email message to your friend that is securely encrypted so that only your friend can read it. To do this, our script uses the RSA algoritm, which is one of the most widely used public-key encryption algorithms.

This notebook will walk you through how to use our script!

First, import the rsa module we wrote, which contains functions to perform RSA.

In [1]:
import rsa

The first thing you would need to send your friend a secure message is to have that friend generate a public and a private key. When they have done that, they should send you the public key and keep their private key a secret. For the purposes of this notebook, we'll just be sending an email to ourselves, so you can go ahead and generate the public and private key yourself.

To generate your keys, you need to first "seed" the algorithm by giving it two random prime integers. For the random prime integers, we set their lower and upper bound to get larger prime integers.

In [8]:
from sympy import randprime

# Generate a large prime number between 10^100 and 10^101
lower_bound = 10**100
upper_bound = 10**101

p = randprime(lower_bound, upper_bound)
print("First prime number:", p)
q = randprime(lower_bound, upper_bound)
while p == q: 
    q = randprime(lower_bound, upper_bound)
print("Second prime number:", q)

First prime number: 77750854433709641040997403337946667601317564653424667380634617490719290142117637779660791320934652339
Second prime number: 19608124183220131873228240734042442250455497635634888551869541919849269223901616390132384905120845577


In [2]:
private_key, public_key = rsa.generate_key(p,q)
print("Public Key:", public_key)
print("Private Key:", private_key)

Public Key: (9754500840991320061643681109611521, 9441118497000238529734018891225837)
Private Key: (9754500840991320061643681109611521, 3393108387771999042425945492137285)


Next, enter a message to send.

Note: The longer your message, the longer your prime numbers have to be to correctly encrypt your message. The provided prime numbers are large enough to encrypt the default "hello" message, but you will need to find larger prime numbers if you want to encrypt a longer message. The next code box will throw an error if your primes are not large enough. In practice, RSA is inefficient for long messages and is generally combined with other techniques to encrypt data.

If you want to find your own primes, try this website, which lets you pick the number of digits you want your prime to have: https://t5k.org/curios/index.php?start=16&stop=19

In [3]:
# Enter a message to encrypt
MESSAGE = "hello"

Next, convert the message to an integer. RSA is a mathematical algorithm that works with numbers, so we need to convert our string message to a number. This function makes use of python's ord() function to do this.

In [5]:
def convert_message_to_int(message):
    """
    Convert message (str) to a single long integer.
    
    Args:
        message: str containing your message

    Returns:
        message_as_int: the message converted to an integer (int)
    """
    # add a one at the beginning to prevent leading zeros from being cut off
    converted_message = "1"

    for i in message:
        int_letter = "{:04d}".format(
            ord(i)
        )  # Convert single letter to zero-padded ASCII
        converted_message += int_letter
    converted_message = int(converted_message)
    return converted_message  # Return as integer



message_as_int = convert_message_to_int(MESSAGE)

print("Your message as an integer:", message_as_int)

# The message must be smaller than p*q to be encrypted correctly
if message_as_int >= p*q:
    raise ValueError("The message in integer form must be smaller than p*q. Choose a shorter message or larger prime numbers.")

Your message as an integer: 101040101010801080111


Now, encrypt the message. To oversimplify a bit, the algorithm takes your message in integer form and raises it to the power of your public key, an operation that is computationally difficult to reverse. That is, unless you have the private key, which you can think about as the inverse of the public key. There's some other fancy math that rsa does, but that's beyond the scope of this notebook.

In [6]:
encrypted_message = rsa.encrypt(public_key, message_as_int)
encrypted_message = str(encrypted_message) # needs to be a string to send over email, but we'll convert it back to an int afterwards
print(encrypted_message)

8006607196980664483011785397559037


Next, use SMTP to send the message to your friend over email.

This part requires you to set up an app password in your email account, which will let the script access your email in order to send/read the email containing your encrypted message. If you don't want to do this, feel free to skip the next two code blocks and proceed to decrypting your message.

If you would like to send yourself or your friend an email containing your encrypted message, follow the steps below.

1. On your email, or preferably an email you don't care about, enable app passwords and copy your email's app password.
2. Create a file in this folder called .env containing the following:
APP_PASS="your email's app password"
MY_EMAIL="the email you want to send from"
RECEIVER_EMAIL="the email you want to send to"

Note: If you're sending an email to a friend, your friend will also need to set up an app password in order for our script to retrieve the sent message from their email. For now, just set both the sender and receiver as your email address.

Run the following code, then check your email to see if the encrypted message is in your inbox. It should just look like a big number!

In [7]:
import smtplib
from email.mime.text import MIMEText
from dotenv import load_dotenv
import os

# Load variables from the .env file
load_dotenv()

# Access the app password, sender email, and receiver email
app_pass = os.getenv("APP_PASS")  # Retrieve the API key
my_email = os.getenv("MY_EMAIL")  # Retrieve sender email
receiver_email = os.getenv("RECEIVER_EMAIL") # Retrieve receiver email
if not app_pass:
    raise EnvironmentError("APP_PASS not found in .env file or environment variables")
if not my_email:
    raise EnvironmentError("MY_EMAIL not found in .env file or environment variables")
if not receiver_email:
    raise EnvironmentError("RECEIVER_EMAIL not found in .env file or environment variables")


def send_email(message, receiver_email):

    subject = "You got an encrypted message!"
    message = message
    sender_email = my_email
    smtp_server = "smtp.gmail.com"
    smtp_port = 587
    smtp_username = my_email
    smtp_password = app_pass

    msg = MIMEText(message)
    msg["Subject"] = subject
    msg["From"] = sender_email
    msg["To"] = receiver_email

    try:
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls()
            server.login(smtp_username, smtp_password)
            server.sendmail(sender_email, receiver_email, msg.as_string())
        # print("Email notification sent successfully!")
    except Exception as e:
        print(f"Failed to send email notification: {e}")


# Send an email to your recipient containing the encrypted message
send_email(encrypted_message, receiver_email)


Then run the following code block to read the most recent email in your (or your friend's) inbox (should be the one you just sent) and display its content.

In [9]:
import imaplib
import email
from dotenv import load_dotenv
import os

# Access the app password
load_dotenv()
app_pass = os.getenv('APP_PASS')  # Retrieve the API key
my_email = os.getenv('MY_EMAIL')
if not app_pass:
    raise EnvironmentError("APP_PASS not found in .env file or environment variables")
if not my_email:
    raise EnvironmentError("MY_EMAIL not found in .env file or environment variables")


# Gmail IMAP settings
IMAP_SERVER = 'imap.gmail.com'
EMAIL = my_email
PASSWORD = app_pass

def read_emails():
    # Connect to Gmail
    mail = imaplib.IMAP4_SSL(IMAP_SERVER)
    mail.login(EMAIL, PASSWORD)

    # Select the inbox
    mail.select("inbox")

    # Search for all emails
    status, messages = mail.search(None, 'ALL')
    email_ids = messages[0].split()

    for e_id in email_ids[-1:]:  # Fetch the last 1 emails
        status, msg_data = mail.fetch(e_id, '(RFC822)')
        for response_part in msg_data:
            if isinstance(response_part, tuple):
                msg = email.message_from_bytes(response_part[1])

                if msg.is_multipart():
                    for part in msg.walk():
                        if part.get_content_type() == "text/plain":
                            message = part.get_payload(decode=True).decode()
                else:
                    message = msg.get_payload(decode=True).decode()

    mail.logout()

    return message

if __name__ == "__main__":
    received_message = int(read_emails().split()[0])
    print("Received message:", received_message)


Received message: 8006607196980664483011785397559037


##### SKIP TO HERE

If you skipped the email portion, run this code block to set the received_message variable to be the encrypted message you generated earlier.

In [12]:
received_message = int(encrypted_message)
print("Received message:", received_message)

Received message: 8006607196980664483011785397559037


Now to decrypt the message. Remember how I said the private key was the inverse of the public key? Basically, you can take your encrypted message to the power of your private key to turn it back into your original message.

This code block will decrypt your message, convert it back into letters, and display it for you.

In [ ]:
def convert_int_to_message(message_as_int):
    """
    Convert the long integer back to the original message.
    
    Args:
        message_as_int: int representing the encoded message

    Returns:
        message: the decoded string message
    """
    # Convert message_as_int into a string for processing
    message_as_int = str(message_as_int)[1:] # remove leading one
    
    # Convert each string of four numbers back to a letter
    original_message = ""
    for i in range(0, len(message_as_int), 4):
        original_message += chr(int(message_as_int[i : i + 4]))

    return original_message

# Convert string message back to int
received_message = int(received_message)

# Decrypt the message
decrypted_message = rsa.decrypt(private_key, received_message)

# Convert back to letters
decrypted_message = convert_int_to_message(decrypted_message)

print("Decrypted message:", decrypted_message)

Decrypted message: hello


Look at that! You just sent a message that no one, not even your email provider, will be able to intercept and read.